In [8]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.reset_defaults
# sns.set_style(style='darkgrid')
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'size'   : 16}
plt.rc('font', **font)
plt.ion()
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams['figure.figsize'] = (20.0, 10.0)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
cd ..

/Users/jeremymiller/GoogleDrive/Data_Science/Galvanize/dsi-capstone


In [36]:
import src.features.build_features as bf

In [37]:
cd src/

/Users/jeremymiller/GoogleDrive/Data_Science/Galvanize/dsi-capstone/src


## Features in student info

In [5]:
df = pd.read_csv('../data/raw/studentInfo.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
code_module             32593 non-null object
code_presentation       32593 non-null object
id_student              32593 non-null int64
gender                  32593 non-null object
region                  32593 non-null object
highest_education       32593 non-null object
imd_band                31482 non-null object
age_band                32593 non-null object
num_of_prev_attempts    32593 non-null int64
studied_credits         32593 non-null int64
disability              32593 non-null object
final_result            32593 non-null object
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


In [7]:
df['final_result'].unique()

array(['Pass', 'Withdrawn', 'Fail', 'Distinction'], dtype=object)

In [9]:
df = bf.drop_nulls(df)

In [10]:
cols = ['gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability']

In [11]:
df = bf.one_hot(df, cols)

In [14]:
df_trans = bf._encode_target(df)

In [16]:
# df_trans.head()

In [ ]:
model = RandomForestClassifier()

In [ ]:
X = df_trans.drop(['id_student', 'final_result', 'module_completed',
                   'code_module', 'code_presentation'], axis = 1)

In [ ]:
y = df_trans['module_completed']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

## Create final grade

In [ ]:
# join assessments to student assessments

In [39]:
std_asmt_df = pd.read_csv('../data/raw/studentAssessment.csv')

In [40]:
asmt_df = pd.read_csv('../data/raw/assessments.csv')

In [42]:
a = bf._join_asssessments(std_asmt_df, asmt_df)

In [60]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170874 entries, 0 to 173911
Data columns (total 10 columns):
id_assessment        170874 non-null int64
id_student           170874 non-null int64
date_submitted       170874 non-null float64
is_banked            170874 non-null float64
score                170874 non-null float64
code_module          170874 non-null object
code_presentation    170874 non-null object
assessment_type      170874 non-null object
date                 170874 non-null float64
weight               170874 non-null float64
dtypes: float64(5), int64(2), object(3)
memory usage: 14.3+ MB


In [50]:
a.dropna(inplace=True)

In [53]:
a.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391.0,18.0,0.0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400.0,22.0,0.0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604.0,17.0,0.0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885.0,26.0,0.0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053.0,19.0,0.0,79.0,AAA,2013J,TMA,19.0,10.0


In [57]:
a['id_student'] = a['id_student'].astype('int64')

In [70]:
a.rename({'date':'due_date'}, axis = 'columns', inplace=True)

In [71]:
df3 = a

In [72]:
df3.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,due_date,weight
0,1752,11391,18.0,0.0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22.0,0.0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17.0,0.0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26.0,0.0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19.0,0.0,79.0,AAA,2013J,TMA,19.0,10.0


In [73]:
df3['days_submitted_early'] = df3['due_date'] - df3['date_submitted']

In [74]:
df3.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,due_date,weight,days_submitted_early
0,1752,11391,18.0,0.0,78.0,AAA,2013J,TMA,19.0,10.0,1.0
1,1752,28400,22.0,0.0,70.0,AAA,2013J,TMA,19.0,10.0,-3.0
2,1752,31604,17.0,0.0,72.0,AAA,2013J,TMA,19.0,10.0,2.0
3,1752,32885,26.0,0.0,69.0,AAA,2013J,TMA,19.0,10.0,-7.0
4,1752,38053,19.0,0.0,79.0,AAA,2013J,TMA,19.0,10.0,0.0


In [78]:
df3.groupby(by=['id_student', 'code_module', 'code_presentation']).mean()[['score', 'days_submitted_early']].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 25819 entries, (6516, AAA, 2014J) to (2698588, BBB, 2014J)
Data columns (total 2 columns):
score                   25819 non-null float64
days_submitted_early    25819 non-null float64
dtypes: float64(2)
memory usage: 687.0+ KB


## Registrations

In [23]:
r_df = pd.read_csv('../data/raw/studentRegistrations.csv')
s_df = pd.read_csv('../data/raw/studentInfo.csv')

In [30]:
r = pd.merge(s_df, r_df, how='outer', on=['code_module', 'code_presentation',
                                      'id_student']).fillna(value=0)

In [34]:
r.isnull().sum()

code_module             0
code_presentation       0
id_student              0
gender                  0
region                  0
highest_education       0
imd_band                0
age_band                0
num_of_prev_attempts    0
studied_credits         0
disability              0
final_result            0
date_registration       0
date_unregistration     0
dtype: int64